# **Submission 2 Proyek Pengembangan dan Pengoperasian Sistem Machine Learning**

## **Data Preparation (Kaggle)**

### **Dataset Summary**

This dataset contains an [US Airline Passenger Satisfaction Survey](https://www.kaggle.com/datasets/teejmahal20/airline-passenger-satisfaction). 

### **Kaggle Environment Variables**

In [1]:
# install library
import os
import pandas as pd

In [2]:
# API
os.environ['KAGGLE_USERNAME'] = 'nurmuhimawann'
os.environ['KAGGLE_KEY'] = '9ec72152f6aaf09a48de3697d930c08e'

In [3]:
# dataset
!kaggle datasets download -d teejmahal20/airline-passenger-satisfaction -f train.csv

  0% 0.00/2.17M [00:00<?, ?B/s]
100% 2.17M/2.17M [00:00<00:00, 92.6MB/s]


In [4]:
# extract zip
!unzip train.csv.zip -d /content/data

# remove unused files
os.remove('train.csv.zip')

Archive:  train.csv.zip
  inflating: /content/data/train.csv  


In [5]:
# read csv
data = pd.read_csv('/content/data/train.csv')
data.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


## **Setup Environment**

In [1]:
# condacolab
!pip install -q condacolab
import condacolab
condacolab.install()
!conda --version

✨🍰✨ Everything looks OK!
conda 22.11.1


In [ ]:
# create conda environment
!conda create --name mlops-tfx python==3.9.15

In [3]:
# activate environment
!conda activate mlops-tfx


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.




In [ ]:
# install modules
!pip install jupyter scikit-learn tensorflow tfx==1.11.0 flask joblib

## **Library**

In [5]:
# import library
import os
import pandas as pd
from typing import Text
from absl import logging
from tfx.orchestration import metadata, pipeline
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner

In [6]:
# extract modules pipeline components
!unzip /content/airline-passenger-satisfaction.zip

# remove unused files
os.remove('airline-passenger-satisfaction.zip')

Archive:  /content/airline-passenger-satisfaction.zip
   creating: modules/
  inflating: modules/airline_satisfaction_trainer.py  
  inflating: modules/airline_satisfaction_transform.py  
  inflating: modules/airline_satisfaction_tuner.py  
  inflating: modules/components.py   
   creating: monitoring/
  inflating: monitoring/Dockerfile   
  inflating: monitoring/prometheus.yml  
  inflating: .gitignore              
  inflating: Dockerfile              
  inflating: README.md               
  inflating: tfx_request.ipynb       
   creating: config/
  inflating: config/prometheus.config  
   creating: images/


## **Data Loading**

In [7]:
# read csv
pd.reset_option('^display.', silent=True)

DATA_PATH = 'data'
df = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
df.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


**DESCRIPTION OF DATA**

There is the following information about the passengers of some airline:

*   ***Gender***: Gender of the passengers (Female, Male)
*   ***Customer Type***: The customer type (Loyal customer, disloyal customer)
*   ***Age***: The actual age of the passengers
*   ***Type of Travel***: Purpose of the flight of the passengers (Personal Travel, Business Travel)
*   ***Class***: Travel class in the plane of the passengers (Business, Eco, Eco Plus)
*   ***Flight distance***: The flight distance of this journey
*   ***Inflight wifi service***: Satisfaction level of the inflight wifi service (0:Not Applicable;1-5)
*   ***Departure/Arrival time convenient***: Satisfaction level of Departure/Arrival time convenient
*   ***Ease of Online booking***: Satisfaction level of online booking
*   ***Gate location***: Satisfaction level of Gate location
*   ***Food and drink***: Satisfaction level of Food and drink
*   ***Online boarding***: Satisfaction level of online boarding
*   ***Seat comfort***: Satisfaction level of Seat comfort
*   ***Inflight entertainment***: Satisfaction level of inflight entertainment
*   ***On-board service***: Satisfaction level of On-board service
*   ***Leg room service***: Satisfaction level of Leg room service
*   ***Baggage handling***: Satisfaction level of baggage handling
*   ***Check-in service***: Satisfaction level of Check-in service
*   ***Inflight service***: Satisfaction level of inflight service
*   ***Cleanliness***: Satisfaction level of Cleanliness
*   ***Departure Delay in Minutes***: Minutes delayed when departure
*   ***Arrival Delay in Minutes***: Minutes delayed when Arrival
*   ***Satisfaction***: Airline satisfaction level(Satisfaction, neutral or dissatisfaction)

In [8]:
# info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 25 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Unnamed: 0                         103904 non-null  int64  
 1   id                                 103904 non-null  int64  
 2   Gender                             103904 non-null  object 
 3   Customer Type                      103904 non-null  object 
 4   Age                                103904 non-null  int64  
 5   Type of Travel                     103904 non-null  object 
 6   Class                              103904 non-null  object 
 7   Flight Distance                    103904 non-null  int64  
 8   Inflight wifi service              103904 non-null  int64  
 9   Departure/Arrival time convenient  103904 non-null  int64  
 10  Ease of Online booking             103904 non-null  int64  
 11  Gate location                      1039

You may notice the following:

*   The column corresponding to the Arrival Delay in Minutes feature has 310 missing values.
*   The first two features are useless and will not affect the classification, so you should get rid of them.
*   Many columns contain categorical values but are of type **'object'** or **'int64'**. Let's replace this type with a special one designed for storing categorical values.

In [9]:
# label counts
df['satisfaction'].value_counts(normalize=True)

neutral or dissatisfied    0.566667
satisfied                  0.433333
Name: satisfaction, dtype: float64

In [10]:
# function to change columns name
def clean_column_names(df):
    # get columns
    columns = df.columns
    
    # change columns name
    new_columns = [col.lower().replace(' ', '_') for col in columns]
    new_columns = [col.replace('-', '').replace('/', '_') for col in new_columns]
    df.columns = new_columns

    # return values
    return df

# drop columns
df.drop(df.iloc[:, [0, 1]], axis=1, inplace=True)

# replace 0 values with the mode value, because 0 is not applicable
mode_value = df['Inflight wifi service'].mode()[0]
df['Inflight wifi service'].replace(0, mode_value, inplace=True)

# encode
map_label = {'neutral or dissatisfied': 0, 'satisfied': 1}
df['satisfaction'] = df['satisfaction'].replace(map_label)

# handling missing values
df = df.dropna()

# clean columns names
df = clean_column_names(df)

# show dataframes
df.head()

,gender,customer_type,age,type_of_travel,class,flight_distance,inflight_wifi_service,departure_arrival_time_convenient,ease_of_online_booking,gate_location,...,inflight_entertainment,onboard_service,leg_room_service,baggage_handling,checkin_service,inflight_service,cleanliness,departure_delay_in_minutes,arrival_delay_in_minutes,satisfaction
0,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,...,5,4,3,4,4,5,5,25,18.0,0
1,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,...,1,1,5,3,1,4,1,1,6.0,0
2,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,...,5,4,3,4,4,4,5,0,0.0,1
3,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,...,2,2,5,3,1,4,2,11,9.0,0
4,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,...,3,3,4,4,3,3,3,0,0.0,1


In [11]:
# descriptive statistic
df.describe()

,age,flight_distance,inflight_wifi_service,departure_arrival_time_convenient,ease_of_online_booking,gate_location,food_and_drink,online_boarding,seat_comfort,inflight_entertainment,onboard_service,leg_room_service,baggage_handling,checkin_service,inflight_service,cleanliness,departure_delay_in_minutes,arrival_delay_in_minutes,satisfaction
count,103594.000000,103594.000000,103594.00000,103594.000000,103594.000000,103594.000000,103594.000000,103594.000000,103594.000000,103594.000000,103594.000000,103594.000000,103594.000000,103594.000000,103594.000000,103594.000000,103594.000000,103594.000000,103594.000000
mean,39.380466,1189.325202,2.81941,3.060081,2.756984,2.977026,3.202126,3.250497,3.439765,3.358341,3.382609,3.351401,3.631687,3.304323,3.640761,3.286397,14.747939,15.178678,0.433394
std,15.113125,997.297235,1.23882,1.525233,1.398934,1.277723,1.329401,1.349433,1.318896,1.333030,1.288284,1.315409,1.181051,1.265396,1.175603,1.312194,38.116737,38.698682,0.495546
min,7.000000,31.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,27.000000,414.000000,2.00000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,3.000000,3.000000,2.000000,0.000000,0.000000,0.000000
50%,40.000000,842.000000,3.00000,3.000000,3.000000,3.000000,3.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,3.000000,4.000000,3.000000,0.000000,0.000000,0.000000
75%,51.000000,1743.000000,4.00000,4.000000,4.000000,4.000000,4.000000,4.000000,5.000000,4.000000,4.000000,4.000000,5.000000,4.000000,5.000000,4.000000,12.000000,13.000000,1.000000
max,85.000000,4983.000000,5.00000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,1592.000000,1584.000000,1.000000


In [12]:
# save data
DATA_PATH = 'data'

# create new directory to save new csv
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

# export to csv
df.to_csv(os.path.join(DATA_PATH, 'airline_passenger_satisfaction.csv'), index=False)

# remove old dataframe
os.remove('./data/train.csv')

# let's see dataframe
df = pd.read_csv(os.path.join(DATA_PATH, 'airline_passenger_satisfaction.csv'))
df.head()

,gender,customer_type,age,type_of_travel,class,flight_distance,inflight_wifi_service,departure_arrival_time_convenient,ease_of_online_booking,gate_location,...,inflight_entertainment,onboard_service,leg_room_service,baggage_handling,checkin_service,inflight_service,cleanliness,departure_delay_in_minutes,arrival_delay_in_minutes,satisfaction
0,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,...,5,4,3,4,4,5,5,25,18.0,0
1,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,...,1,1,5,3,1,4,1,1,6.0,0
2,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,...,5,4,3,4,4,4,5,0,0.0,1
3,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,...,2,2,5,3,1,4,2,11,9.0,0
4,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,...,3,3,4,4,3,3,3,0,0.0,1


## **Set Pipeline Variable**

In [13]:
# pipeline name
PIPELINE_NAME = 'airline-satisfaction-pipeline'

# pipeline inputs
DATA_ROOT = 'data'
TRANSFORM_MODULE_FILE = 'modules/airline_satisfaction_transform.py'
TUNER_MODULE_FILE = 'modules/airline_satisfaction_tuner.py'
TRAINER_MODULE_FILE = 'modules/airline_satisfaction_trainer.py'

# pipeline outputs
OUTPUT_BASE = 'outputs'

serving_model_dir = os.path.join(OUTPUT_BASE, 'serving_model')
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, 'metadata.sqlite')

## **ML Pipelines (Pipeline Orchestrator)**

In [14]:
# build TFX component
def init_local_pipeline(
    components, pipeline_root: Text
) -> pipeline.Pipeline:
    """Init local pipeline

    Args:
        components (dict): tfx components
        pipeline_root (Text): path to pipeline directory

    Returns:
        pipeline.Pipeline: apache beam pipeline orchestration
    """
    logging.info(f"Pipeline root set to: {pipeline_root}")
    beam_args = [
        '--direct_running_mode=multi_processing'
        # 0 auto-detect based on on the number of CPUs available
        # during execution time.
        '----direct_num_workers=0'
    ]

    return pipeline.Pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=pipeline_root,
        components=components,
        enable_cache=True,
        metadata_connection_config=metadata.sqlite_metadata_connection_config(
            metadata_path
        ),
        eam_pipeline_args=beam_args
    )

In [15]:
# running pipelines
from modules import components

logging.set_verbosity(logging.INFO)

components = components.init_components({
    'data_dir': DATA_ROOT,
    'transform_module': TRANSFORM_MODULE_FILE,
    'tuner_module': TUNER_MODULE_FILE,
    'training_module': TRAINER_MODULE_FILE,
    'training_steps': 5000,
    'eval_steps': 1000,
    'serving_model_dir': serving_model_dir
})

pipeline = init_local_pipeline(components, pipeline_root)
BeamDagRunner().run(pipeline=pipeline)

Trial 30 Complete [00h 05m 47s]
val_loss: 0.10664737224578857

Best val_loss So Far: 0.10023172199726105
Total elapsed time: 01h 14m 34s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit
INFO:absl:Finished tuning... Tuner ID: tuner0
INFO:absl:Best HyperParameters: {'space': [{'class_name': 'Choice', 'config': {'name': 'num_layers', 'default': 1, 'conditions': [], 'values': [1, 2, 3], 'ordered': True}}, {'class_name': 'Int', 'config': {'name': 'dense_units', 'default': None, 'conditions': [], 'min_value': 16, 'max_value': 256, 'step': 16, 'sampling': None}}, {'class_name': 'Float', 'config': {'name': 'dropout_rate', 'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.8, 'step': 0.1, 'sampling': None}}, {'class_name': 'Choice', 'config': {'name': 'learning_rate', 'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}}], 'values': {'num_layers': 1, 'dense_units': 80, 'dropout_rate': 0.30000000000000004, 'learning_rate': 0.001, 'tuner/epochs': 10, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}}
INFO:absl:Best Hyperparameters are written to outputs/airline-satisfaction-pipeli

Results summary
Results in outputs/airline-satisfaction-pipeline/Tuner/.system/executor_execution/7/.temp/7/airline_satisfaction_kt
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 1
dense_units: 80
dropout_rate: 0.30000000000000004
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.10023172199726105
Trial summary
Hyperparameters:
num_layers: 2
dense_units: 144
dropout_rate: 0.30000000000000004
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0022
Score: 0.10031027346849442
Trial summary
Hyperparameters:
num_layers: 1
dense_units: 240
dropout_rate: 0.2
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0012
Score: 0.1006188839673996
Trial summary
Hyperparameters:
num_layers: 2
dense_units: 160
dropout_rate: 0.7000000000000001
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 1


INFO:absl:node Trainer is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "tfx.components.trainer.component.Trainer"
    base_type: TRAIN
  }
  id: "Trainer"
}
contexts {
  contexts {
    type {
      name: "pipeline"
    }
    name {
      field_value {
        string_value: "airline-satisfaction-pipeline"
      }
    }
  }
  contexts {
    type {
      name: "pipeline_run"
    }
    name {
      field_value {
        string_value: "20230426-050729.007958"
      }
    }
  }
  contexts {
    type {
      name: "node"
    }
    name {
      field_value {
        string_value: "airline-satisfaction-pipeline.Trainer"
      }
    }
  }
}
inputs {
  inputs {
    key: "examples"
    value {
      channels {
        producer_node_query {
          id: "Transform"
        }
        context_queries {
          type {
            name: "pipeline"
          }
          name {
            field_value {
              string_value: "airline-satisfaction-pipeline"
            }

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 gender_xf (InputLayer)         [(None, 3)]          0           []                               
                                                                                                  
 customer_type_xf (InputLayer)  [(None, 3)]          0           []                               
                                                                                                  
 type_of_travel_xf (InputLayer)  [(None, 3)]         0           []                               
                                                                                                  
 class_xf (InputLayer)          [(None, 4)]          0           []                               
                                                                                            

...

INFO:tensorflow:Assets written to: outputs/airline-satisfaction-pipeline/Trainer/model/8/Format-Serving/assets
INFO:absl:Training complete. Model written to outputs/airline-satisfaction-pipeline/Trainer/model/8/Format-Serving. ModelRun written to outputs/airline-satisfaction-pipeline/Trainer/model_run/8
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 8 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'model_run': [Artifact(artifact: uri: "outputs/airline-satisfaction-pipeline/Trainer/model_run/8"
, artifact_type: name: "ModelRun"
)], 'model': [Artifact(artifact: uri: "outputs/airline-satisfaction-pipeline/Trainer/model/8"
, artifact_type: name: "Model"
base_type: MODEL
)]}) for execution 8
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:node Trainer is finished.
INFO:absl:node Evaluator is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "tfx.components

INFO:absl:The 'example_splits' parameter is not set, using 'eval' split.
INFO:absl:Evaluating model.
INFO:absl:udf_utils.get_fn {'eval_config': '{\n  "metrics_specs": [\n    {\n      "metrics": [\n        {\n          "class_name": "AUC"\n        },\n        {\n          "class_name": "Precision"\n        },\n        {\n          "class_name": "Recall"\n        },\n        {\n          "class_name": "ExampleCount"\n        },\n        {\n          "class_name": "TruePositives"\n        },\n        {\n          "class_name": "FalsePositives"\n        },\n        {\n          "class_name": "TrueNegatives"\n        },\n        {\n          "class_name": "FalseNegatives"\n        },\n        {\n          "class_name": "BinaryAccuracy",\n          "threshold": {\n            "change_threshold": {\n              "absolute": 0.0001,\n              "direction": "HIGHER_IS_BETTER"\n            },\n            "value_threshold": {\n              "lower_bound": 0.5\n            }\n          }\n  

INFO:absl:Evaluation complete. Results written to outputs/airline-satisfaction-pipeline/Evaluator/evaluation/9.
INFO:absl:Checking validation results.


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
INFO:absl:Blessing result True written to outputs/airline-satisfaction-pipeline/Evaluator/blessing/9.
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 9 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'evaluation': [Artifact(artifact: uri: "outputs/airline-satisfaction-pipeline/Evaluator/evaluation/9"
, artifact_type: name: "ModelEvaluation"
)], 'blessing': [Artifact(artifact: uri: "outputs/airline-satisfaction-pipeline/Evaluator/blessing/9"
, artifact_type: name: "ModelBlessing"
)]}) for execution 9
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:node Evaluator is finished.
INFO:absl:node Pusher is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "tfx.components.pusher.component.Pusher"
    base_type: DEPLOY
  }
  id: "Pusher"
}
contexts {
  contexts {
    type {
     

## **Export Model**

In [16]:
# export model
!zip -r data.zip data/
!zip -r images.zip images/
!zip -r outputs.zip outputs/
!pip freeze > requirements.txt

  adding: data/ (stored 0%)
  adding: data/airline_passenger_satisfaction.csv (deflated 84%)
  adding: images/ (stored 0%)
  adding: images/model_plot.png (deflated 19%)
  adding: outputs/ (stored 0%)
  adding: outputs/airline-satisfaction-pipeline/ (stored 0%)
  adding: outputs/airline-satisfaction-pipeline/StatisticsGen/ (stored 0%)
  adding: outputs/airline-satisfaction-pipeline/StatisticsGen/statistics/ (stored 0%)
  adding: outputs/airline-satisfaction-pipeline/StatisticsGen/statistics/3/ (stored 0%)
  adding: outputs/airline-satisfaction-pipeline/StatisticsGen/statistics/3/Split-eval/ (stored 0%)
  adding: outputs/airline-satisfaction-pipeline/StatisticsGen/statistics/3/Split-eval/FeatureStats.pb (deflated 79%)
  adding: outputs/airline-satisfaction-pipeline/StatisticsGen/statistics/3/Split-train/ (stored 0%)
  adding: outputs/airline-satisfaction-pipeline/StatisticsGen/statistics/3/Split-train/FeatureStats.pb (deflated 78%)
  adding: outputs/airline-satisfaction-pipeline/Statist

## **Deactivate Virtual Environment**

In [17]:
# deactivate an active environment
!conda deactivate


CommandNotFoundError: Your shell has not been properly configured to use 'conda deactivate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.


